The code below executes a portfolio which is 50% absolute momnetum and 50% absolute reversion. It implements the strategy for a set of Alpaca borkerage accounts with API keys stored in a firestore database. Descriptions of the functions are made in code comments


In [ ]:
!pip install fix-yahoo-finance==0.1.30
!pip install yfinance
!pip install --upgrade firebase-admin

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 16, 8
import yfinance as yf

#requirements.txt
# Function dependencies, for example:
# package>=version
'''numpy
pandas
pandas_datareader
fix-yahoo-finance==0.1.30
yfinance
firebase-admin'''



In [ ]:

import pandas as pd
import numpy as np
import yfinance as yf

#DEFINE FUNCTIONS

def get_current_portfolio_AUM(URL, ID, SECRET):
  '''use REST API to get current AUM'''
  import requests
  headers = {'APCA-API-KEY-ID': ID , 'APCA-API-SECRET-KEY': SECRET}
  r = requests.get(url = URL+"/v2/account", headers = headers)
  data_bytes = r.content
  data_string = data_bytes.decode('utf-8')
  import json
  data_dict = json.loads(data_string)
  portfolio_AUM = int(float(data_dict['portfolio_value']))
  return portfolio_AUM


def set_initial_trade(URL, ID, SECRET, portfolio_AUM):
  '''clear all portfolio positions and set portfolio initialisation positions'''
  #initialise access tools
  import requests
  headers = {'APCA-API-KEY-ID': ID , 'APCA-API-SECRET-KEY': SECRET}

  #clear existing orders
  r = requests.delete(url = URL+"/v2/orders", headers = headers)
  data_bytes = r.content
  data_string = data_bytes.decode('utf-8')
  import json
  data_dict = json.loads(data_string)
  print('clear orders response: ', data_dict)

  #clear existing positions
  r = requests.delete(url = URL+"/v2/positions", headers = headers)
  data_bytes = r.content
  data_string = data_bytes.decode('utf-8')
  import json
  data_dict = json.loads(data_string)
  print('clear positions response: ', data_dict)

  #set initial trade
  trade_dict = {
      "IEF": portfolio_AUM/6,
      "TLT": portfolio_AUM/3,
      "QQQ": portfolio_AUM/6,
      "SPY": portfolio_AUM/6,
      "BRK.B": portfolio_AUM/6,
                }

  status = "success"
  for symbol in ['IEF', 'TLT','QQQ','SPY', 'BRK.B']:
    if trade_dict[symbol] > 0:
      params = {"symbol": symbol, "notional": int(np.abs(trade_dict[symbol])), "side": "buy", "type": "market", "time_in_force": "day" }
    else:
      params = {"symbol": symbol, "notional": int(np.abs(trade_dict[symbol])), "side": "sell", "type": "market", "time_in_force": "day" }
    try:
      r = requests.post(url = URL+"/v2/orders", headers = headers, json = params)
      data_bytes = r.content
      data_string = data_bytes.decode('utf-8')
      import json
      data_dict = json.loads(data_string)
      print(data_dict["status"])
    except:
      print(data_dict["message"])
      status = "error"
  import time
  time.sleep(10)

  return status


def get_users():
  import firebase_admin
  from firebase_admin import credentials
  from firebase_admin import firestore


  # Use a service account
  cred = credentials.Certificate('/content/lacuna-momrev-df90e9b2d018.json')
  try:
    firebase_admin.initialize_app(cred)
  except:
    print('error occurred in initialising firebase. Perhaps already initialised')

  db = firestore.client()

  users_ref = db.collection(u'users')
  docs = users_ref.stream()

  uid_ = []
  secret_ = []
  url_ = []

  for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')
    id = doc.id
    doc = doc.to_dict()
    print(doc['name'])
    if doc['status'] == "active":
      uid_.append(doc['ID'])
      secret_.append(doc['SECRET'])
      url_.append(doc['URL'])
    elif doc['status'] == "subscribed":
      portfolio_AUM = get_current_portfolio_AUM(doc['URL'], doc['ID'], doc['SECRET'])
      set_initial_trade(doc['URL'], doc['ID'], doc['SECRET'],portfolio_AUM)
      doc_ref = db.collection(u'users').document(id)
      doc_ref.update({u'status': 'active'})
    else:
      print("Unsubscribed customer. Skip...")


  return uid_, secret_, url_


def get_stocks_alloc(URL, ID, SECRET):
  '''use REST API to get current dollar allocations'''
  import requests
  headers = {'APCA-API-KEY-ID': ID , 'APCA-API-SECRET-KEY': SECRET}
  r = requests.get(url = URL+"/v2/positions/SPY", headers = headers)
  data_bytes = r.content
  data_string = data_bytes.decode('utf-8')
  import json
  data_dict = json.loads(data_string)
  spy_dol = int(float(data_dict['market_value']))
  stock_dol = spy_dol * 3
  return stock_dol
  #return 50000

def get_bonds_alloc(URL, ID, SECRET):
  '''use REST API to get current dollar allocations'''

  import requests
  headers = {'APCA-API-KEY-ID': ID , 'APCA-API-SECRET-KEY': SECRET}
  r = requests.get(url = URL+"/v2/positions/IEF", headers = headers)
  data_bytes = r.content
  data_string = data_bytes.decode('utf-8')
  import json
  data_dict = json.loads(data_string)
  ief_dol = int(float(data_dict['market_value']))
  bonds_dol = ief_dol * 3
  return bonds_dol
  #return 50000

def get_current_perc_positions(portfolio_AUM, stock_dol, bond_dol):
  '''calculate percentage positions from dollar allocations and AUM'''
  stock_curp = float(stock_dol)/portfolio_AUM
  bond_curp = float(bond_dol)/portfolio_AUM
  return stock_curp, bond_curp

def reduce_current_portfolio(stock_curp, bond_curp):
  '''reduce the current portfolio percentage allocations to 85%'''
  stock_red = .85 * stock_curp
  bond_red = .85 * bond_curp
  return stock_red, bond_red

def get_stock_bond_series():
  '''get stock/bonds series from yfinance'''
  start= '2021-1-1'
  end = '2023-1-30'
  ohlc_selection = 'Adj Close'
  spy = 1* yf.download('spy', start=start, end=end)[ohlc_selection].pct_change()[1:].fillna(0.000);
  qqq = 1* yf.download("qqq", start=start, end=end)[ohlc_selection].pct_change()[1:].fillna(0.000);
  brk = 1* yf.download("brk-b", start=start, end=end)[ohlc_selection].pct_change()[1:].fillna(0.000);
  ief = 1* yf.download("ief", start=start, end=end)[ohlc_selection].pct_change()[1:].fillna(0.000);
  tlt = 1* yf.download("tlt", start=start, end=end)[ohlc_selection].pct_change()[1:].fillna(0.000);
  stock_ser = 0.333*qqq + 0.333*spy + 0.333*brk
  bond_ser = 0.333*ief + 0.666*tlt
  return stock_ser, bond_ser

def momentum(stock_ser, bond_ser):
  '''run the momentum strategy for tomorrow's position'''
  if np.prod(1+stock_ser[-252:]) - np.prod(1+stock_ser[-273:-21]) > 0:
    stock_mom = 1
    bond_mom = 0
  else:
    stock_mom = 0
    bond_mom = 1
  return stock_mom, bond_mom

def reversion(stock_ser, bond_ser):
  '''run the reversion strategy for tomorrow's position'''
  if np.prod(1+stock_ser[-65:]) - np.prod(1+stock_ser[-70:-5]) > 0:
    stock_rev = 0
    bond_rev = 1
  else:
    stock_rev = 1
    bond_rev = 0
  return stock_rev, bond_rev

def calc_new_perc_alocations(stock_red, bond_red, stock_mom, bond_mom, stock_rev, bond_rev):
  '''calculate the new percentage allocations from current, momentum, and reversal'''
  stock_newp = stock_red + 0.05*stock_mom + 0.1*stock_rev
  bond_newp = bond_red + 0.05*bond_mom + 0.1*bond_rev
  return stock_newp, bond_newp

def calc_perc_differential(stock_newp, bond_newp, stock_curp, bond_curp):
  '''calculate the percentage differential between current and new'''
  stock_diff = stock_newp - stock_curp
  bond_diff = bond_newp - bond_curp
  return stock_diff, bond_diff

def calc_dollar_differential(stock_diff, bond_diff, portfolio_AUM):
  '''calculate the dollar differential (trades) from the percentage differential and the AUM'''
  stock_trade = stock_diff * portfolio_AUM
  bond_trade = bond_diff * portfolio_AUM
  return stock_trade, bond_trade

def execute_trades(URL, ID, SECRET, stock_trade, bond_trade, leverage= 1):
  '''calculate and execute the trades on representative assets'''
  ief_trade = bond_trade/3
  tlt_trade = bond_trade/1.5# - ief_trade
  spy_trade = stock_trade/3
  qqq_trade = stock_trade/3
  brk_trade = stock_trade/3# - (spy_trade + qqq_trade)
  print(ief_trade, tlt_trade, spy_trade, qqq_trade, brk_trade)
  trade_dict = {
      "IEF": ief_trade,
      "TLT": tlt_trade,
      "QQQ": qqq_trade,
      "SPY": spy_trade,
      "BRK.B": brk_trade,
                }
  import requests
  headers = {'APCA-API-KEY-ID': ID , 'APCA-API-SECRET-KEY': SECRET}
  for symbol in ['IEF', 'TLT','QQQ','SPY', 'BRK.B']:
    if trade_dict[symbol] > 0:
      params = {"symbol": symbol, "notional": int(np.abs(trade_dict[symbol])), "side": "buy", "type": "market", "time_in_force": "day" }
    else:
      params = {"symbol": symbol, "notional": int(np.abs(trade_dict[symbol])), "side": "sell", "type": "market", "time_in_force": "day" }
    try:
      r = requests.post(url = URL+"/v2/orders", headers = headers, json = params)
      data_bytes = r.content
      data_string = data_bytes.decode('utf-8')
      import json
      data_dict = json.loads(data_string)
      print(data_dict["status"])
      status = "completed"
    except:
      print("ERROR")
      status = "error"
  return status





In [ ]:
#EXECUTION SCRIPT
status = "initiated but not completed. Unsubscribed customer?"
id_, secret_, url_ = get_users()

for i in range(len(url_)):
  URL = url_[0]
  ID = id_[0]
  SECRET = secret_[0]
  print(ID,SECRET)

  portfolio_AUM = get_current_portfolio_AUM(URL, ID, SECRET)
  print(portfolio_AUM)
  stock_dol = get_stocks_alloc(URL, ID, SECRET)
  bond_dol = get_bonds_alloc(URL, ID, SECRET)
  print("stock current dollar: ", stock_dol)
  print("bond current dollar: ", bond_dol)
  stock_curp, bond_curp = get_current_perc_positions(portfolio_AUM, stock_dol, bond_dol)
  print("stock current %: ", stock_curp)
  print("bond current %: ", bond_curp)
  stock_red, bond_red = reduce_current_portfolio(stock_curp, bond_curp)
  stock_ser, bond_ser = get_stock_bond_series()
  stock_mom, bond_mom = momentum(stock_ser, bond_ser)
  print('stock/bond momentum: ', stock_mom, bond_mom)
  stock_rev, bond_rev = reversion(stock_ser, bond_ser)
  print('stock/bond reversion: ', stock_mom, bond_mom)
  stock_newp, bond_newp = calc_new_perc_alocations(stock_red, bond_red, stock_mom, bond_mom, stock_rev, bond_rev)
  print("stock/bond new %: ", stock_newp, bond_newp)
  stock_diff, bond_diff = calc_perc_differential(stock_newp, bond_newp, stock_curp, bond_curp)
  print("stock current %diff: ", stock_diff)
  print("stock current %diff: ", bond_diff)
  stock_trade, bond_trade = calc_dollar_differential(stock_diff, bond_diff, portfolio_AUM)
  print(stock_trade, bond_trade)
  status = execute_trades(URL, ID, SECRET, stock_trade, bond_trade, leverage= 1)
  print(status)
